# Case Study #2: Pizza Runner [Website Link](https://8weeksqlchallenge.com/case-study-2/)

Common Table Expressions or CTEs act like *temporary views* that exist only for the duration of a single SQL statement. 

There are two kinds of common table expressions: "ordinary" and "recursive". 

**Ordinary common table expressions** are helpful for making queries easier to understand by factoring subqueries out of the main SQL statement. 

**Recursive common table expressions** provide the ability to do hierarchical or recursive queries of trees and graphs, a capability that is not otherwise available in the SQL language.

[Source](https://www.sqlite.org/lang_with.html)

The **UNION** command combines the result set of two or more SELECT statements (only distinct values) [Source](https://www.w3schools.com/sql/sql_ref_union.asp)

The **UNION ALL** command combines the result set of two or more SELECT statements (allows duplicate values). [Source](https://www.w3schools.com/sql/sql_ref_union_all.asp)

In [1]:
%load_ext sql
%sql sqlite:///pizza_runner.db

In [2]:
%%sql
SELECT *
FROM customer_orders 
--pizza_toppings

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,pizza_id,exclusions,extras,order_time
1,101,1,,,2020-01-01 18:05:02.000000
2,101,1,,,2020-01-01 19:00:52.000000
3,102,1,,,2020-01-02 23:51:23.000000
3,102,2,,None,2020-01-02 23:51:23.000000
4,103,1,4,,2020-01-04 13:23:46.000000
4,103,1,4,,2020-01-04 13:23:46.000000
4,103,2,4,,2020-01-04 13:23:46.000000
5,104,1,null,1,2020-01-08 21:00:29.000000
6,101,2,null,null,2020-01-08 21:03:13.000000
7,105,2,null,1,2020-01-08 21:20:29.000000


# A. Pizza Metrics

## Question A1: How many pizzas were ordered?

From case: "Customer pizza orders are captured in the customer_orders table with 1 row for each individual pizza that is part of the order."

In [3]:
%%sql 
SELECT COUNT(*) as num_pizzas
FROM customer_orders;

 * sqlite:///pizza_runner.db
Done.


num_pizzas
14


## Question A2: How many unique customer orders were made?

In [4]:
%%sql 
SELECT COUNT(DISTINCT(order_id)) as unique_customer_orders
FROM customer_orders;

 * sqlite:///pizza_runner.db
Done.


unique_customer_orders
10


## Question A3: How many successful orders were delivered by each runner?

In [5]:
# %%sql 
# SELECT *
# FROM customer_orders AS cust
# LEFT JOIN runner_orders AS run ON cust.order_id = run.order_id
# WHERE run.pickup_time IS NOT 'null';

In [6]:
%%sql 
SELECT run.runner_id, COUNT(DISTINCT(cust.order_id)) as orders
FROM customer_orders AS cust
LEFT JOIN runner_orders AS run ON cust.order_id = run.order_id
WHERE run.pickup_time IS NOT 'null'
GROUP BY run.runner_id;

 * sqlite:///pizza_runner.db
Done.


runner_id,orders
1,4
2,3
3,1


## Question A4: How many of each type of pizza was delivered?

In [7]:
%%sql 
SELECT customer_orders.pizza_id, COUNT(*) as num_pizzas
FROM customer_orders
LEFT JOIN runner_orders AS run ON customer_orders.order_id = run.order_id
WHERE run.pickup_time IS NOT 'null'
GROUP BY pizza_id;

 * sqlite:///pizza_runner.db
Done.


pizza_id,num_pizzas
1,9
2,3


## Question A5: How many Vegetarian and Meatlovers were ordered by each customer?

In [8]:
%%sql 
SELECT customer_orders.customer_id, pizza_names.pizza_name, COUNT(*) as num_pizzas
FROM customer_orders
LEFT JOIN pizza_names ON customer_orders.pizza_id = pizza_names.pizza_id
GROUP BY customer_id, pizza_name

 * sqlite:///pizza_runner.db
Done.


customer_id,pizza_name,num_pizzas
101,Meatlovers,2
101,Vegetarian,1
102,Meatlovers,2
102,Vegetarian,1
103,Meatlovers,3
103,Vegetarian,1
104,Meatlovers,3
105,Vegetarian,1


## Question A6: What was the maximum number of pizzas delivered in a single order?

In [9]:
%%sql 
SELECT order_id, COUNT(order_id) as pizza_per_order
FROM customer_orders
GROUP BY order_id
ORDER BY pizza_per_order DESC
LIMIT 1;

 * sqlite:///pizza_runner.db
Done.


order_id,pizza_per_order
4,3


## Question A7: For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [10]:
%%sql 
SELECT cust.order_id, cust.customer_id, cust.exclusions, cust.extras
FROM customer_orders as cust
LEFT JOIN runner_orders AS run ON cust.order_id = run.order_id
WHERE run.pickup_time IS NOT 'null' AND
    (cust.exclusions IS NOT '' AND cust.exclusions IS NOT 'null') OR (cust.extras IS NOT 'null' AND cust.extras IS NOT NULL AND cust.extras IS NOT '')

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,exclusions,extras
4,103,4,
4,103,4,
4,103,4,
5,104,null,1
7,105,null,1
9,103,4,"1, 5"
10,104,"2, 6","1, 4"


In [11]:
# 6 pizzas with at least one change (exclusion or extras)

In [12]:
%%sql 
SELECT cust.order_id, cust.customer_id, cust.exclusions, cust.extras
FROM customer_orders as cust
LEFT JOIN runner_orders AS run ON cust.order_id = run.order_id
WHERE run.pickup_time IS NOT 'null' AND
    (cust.exclusions IS '' OR cust.exclusions IS 'null') AND (cust.extras IS 'null' OR cust.extras IS NULL OR cust.extras IS '')

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,exclusions,extras
1,101,,
2,101,,
3,102,,
3,102,,None
8,102,null,null
10,104,null,null


In [13]:
# six pizzas with no changes (exclusion or extras)

## Question A8: How many pizzas were delivered that had both exclusions and extras?

In [14]:
%%sql 
SELECT cust.order_id, cust.customer_id, cust.exclusions, cust.extras
FROM customer_orders as cust
LEFT JOIN runner_orders AS run ON cust.order_id = run.order_id
WHERE run.pickup_time IS NOT 'null' AND
    (cust.exclusions IS NOT '' AND cust.exclusions IS NOT 'null') AND (cust.extras IS NOT 'null' AND cust.extras IS NOT NULL AND cust.extras IS NOT '')

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,exclusions,extras
10,104,"2, 6","1, 4"


In [15]:
#only 1 pizza has both exclussions and extras

## Question A9: What was the total volume of pizzas ordered for each hour of the day?

In [16]:
%%sql 
SELECT strftime('%H', order_time) as hour, COUNT(*) as num_pizzas
FROM customer_orders
GROUP BY hour
ORDER BY hour ASC;

 * sqlite:///pizza_runner.db
Done.


hour,num_pizzas
11,1
13,3
18,3
19,1
21,3
23,3


## Question A10: What was the volume of orders for each day of the week?

In [17]:
#--'%w' day of week 0-6 with Sunday==0

In [18]:
%%sql 
SELECT strftime('%w', order_time) as day_of_week, COUNT(*) as num_pizzas
FROM customer_orders
GROUP BY day_of_week
ORDER BY day_of_week ASC;

 * sqlite:///pizza_runner.db
Done.


day_of_week,num_pizzas
3,5
4,3
5,1
6,5


# B. Runner and Customer Experience

## Question B1: How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [19]:
%%sql
SELECT strftime('%W', registration_date) as week_num, COUNT(*) as num_runners
FROM runners
GROUP BY week_num
ORDER BY week_num ASC;

 * sqlite:///pizza_runner.db
Done.


week_num,num_runners
00,2
01,1
02,1


## Question B2: What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [20]:
## not quite sure I understand what data is needed to answer this question

## Question B3: Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [21]:
# -- IN PROGRESS

In [22]:
%%sql

SELECT COUNT(*) as num_pizzas, customer_orders.order_id, customer_orders.customer_id, customer_orders.order_time, 
    runner_orders.*, 
    strftime('%M:%S', strftime('%Y-%m-%d %H:%M:%S', runner_orders.pickup_time) - strftime('%Y-%m-%d %H:%M:%S', customer_orders.order_time)) as prep_time
FROM customer_orders
LEFT JOIN runner_orders ON customer_orders.order_id = runner_orders.order_id
WHERE runner_orders.pickup_time IS NOT 'null'
GROUP BY customer_orders.order_id;

 * sqlite:///pizza_runner.db
Done.


num_pizzas,order_id,customer_id,order_time,order_id_1,runner_id,pickup_time,distance,duration,cancellation,prep_time
1,1,101,2020-01-01 18:05:02.000000,1,1,2020-01-01 18:15:34,20km,32 minutes,,00:00
1,2,101,2020-01-01 19:00:52.000000,2,1,2020-01-01 19:10:54,20km,27 minutes,,00:00
2,3,102,2020-01-02 23:51:23.000000,3,1,2020-01-03 00:12:37,13.4km,20 mins,None,00:00
3,4,103,2020-01-04 13:23:46.000000,4,2,2020-01-04 13:53:03,23.4,40,None,00:00
1,5,104,2020-01-08 21:00:29.000000,5,3,2020-01-08 21:10:57,10,15,None,00:00
1,7,105,2020-01-08 21:20:29.000000,7,2,2020-01-08 21:30:45,25km,25mins,null,00:00
1,8,102,2020-01-09 23:54:33.000000,8,2,2020-01-10 00:15:02,23.4 km,15 minute,null,00:00
2,10,104,2020-01-11 18:34:49.000000,10,1,2020-01-11 18:50:20,10km,10minutes,null,00:00


## Question B4: What was the average distance travelled for each customer?

In [23]:
%%sql
WITH temp AS (
    SELECT order_id, customer_id, COUNT(*) AS num_pizza
    FROM customer_orders
    GROUP BY order_id
)
SELECT temp.customer_id, AVG(runner_orders.distance) as avg_distance
FROM runner_orders
JOIN temp ON runner_orders.order_id = temp.order_id
WHERE runner_orders.pickup_time IS NOT 'null'
GROUP BY temp.customer_id;

 * sqlite:///pizza_runner.db
Done.


customer_id,avg_distance
101,20.0
102,18.4
103,23.4
104,10.0
105,25.0


## Question B5: What was the difference between the longest and shortest delivery times for all orders?

In [24]:
%%sql
SELECT MAX(TRIM(REPLACE(distance, 'km', ''))) - MIN(TRIM(REPLACE(distance, 'km', ''))) as diff_between_longest_and_shortest_delivery
FROM runner_orders
WHERE pickup_time IS NOT 'null';

 * sqlite:///pizza_runner.db
Done.


diff_between_longest_and_shortest_delivery
15


## Question B6: What was the average **speed** for **each** runner for **each** delivery and do you notice any trend for these values?

In [25]:
# %%sql
# SELECT  *
# FROM runner_orders

# /*IN PROGRESS*/

In [26]:
##av.note:TODO: figure out why some values are zero when they should not be... also convert from km/min to mph

In [27]:
%%sql
SELECT runner_id, order_id, distance, duration, cancellation, pickup_time,
    TRIM(REPLACE(distance, 'km', '')) as distance_km, 
    TRIM(REPLACE(REPLACE(REPLACE(runner_orders.duration, 'minutes', ''), 'mins', ''), 'minute', '')) as duration_min, 
    (CAST(TRIM(REPLACE(distance, 'km', '')) AS numeric) / 
     CAST(TRIM(REPLACE(REPLACE(REPLACE(runner_orders.duration, 'minutes', ''), 'mins', ''), 'minute', ''))AS numeric) ) as km_per_min
FROM runner_orders
WHERE pickup_time IS NOT 'null'
GROUP BY order_id, runner_id
ORDER BY runner_id, duration;

 * sqlite:///pizza_runner.db
Done.


runner_id,order_id,distance,duration,cancellation,pickup_time,distance_km,duration_min,km_per_min
1,10,10km,10minutes,null,2020-01-11 18:50:20,10,10,1
1,3,13.4km,20 mins,None,2020-01-03 00:12:37,13.4,20,0.67
1,2,20km,27 minutes,,2020-01-01 19:10:54,20,27,0
1,1,20km,32 minutes,,2020-01-01 18:15:34,20,32,0
2,8,23.4 km,15 minute,null,2020-01-10 00:15:02,23.4,15,1.5599999999999998
2,7,25km,25mins,null,2020-01-08 21:30:45,25,25,1
2,4,23.4,40,None,2020-01-04 13:53:03,23.4,40,0.585
3,5,10,15,None,2020-01-08 21:10:57,10,15,0


In [28]:
##av.note: it seems like runners are driving are fairly low speeds (around 25 mph), one runner (id 2) for one delivery (id 8) was driving very fast (avg. 58 mph)

## Question B7: What is the successful delivery percentage for each runner?

In [29]:
%%sql
SELECT runner_id, ROUND(AVG(CASE WHEN pickup_time is NOT 'null' THEN 1 ELSE 0 END) *100, 2) AS percent_successful
FROM runner_orders
GROUP BY runner_id;

 * sqlite:///pizza_runner.db
Done.


runner_id,percent_successful
1,100.0
2,75.0
3,50.0


# C. Ingredient Optimisation

In [30]:
## will need to use recrusive CTE in this ection, in other SQL dialects, could use STR_SPLIT

## Question C1: What are the standard ingredients for each pizza?

In [31]:
%%sql
SELECT *
FROM pizza_recipes

-- IN PROGRESS

 * sqlite:///pizza_runner.db
Done.


pizza_id,toppings
1,"1, 2, 3, 4, 5, 6, 8, 10"
2,"4, 6, 7, 9, 11, 12"


In [32]:
# https://stackoverflow.com/questions/24258878/how-to-split-comma-separated-value-in-sqlite
# WITH split(word, str) AS (
#     -- alternatively put your query here
#     -- SELECT '', category||' ' FROM categories
#     SELECT '', 'Auto A 1234444'||' '
#     UNION ALL SELECT
#     substr(str, 0, instr(str, ' ')),
#     substr(str, instr(str, ' ')+1)
#     FROM split WHERE str!=''
# ) SELECT word FROM split WHERE word!='';

In [33]:
%%sql
SELECT *
FROM pizza_toppings

 * sqlite:///pizza_runner.db
Done.


topping_id,topping_name
1,Bacon
2,BBQ Sauce
3,Beef
4,Cheese
5,Chicken
6,Mushrooms
7,Onions
8,Pepperoni
9,Peppers
10,Salami


## Question C2: What was the most commonly added extra?

In [34]:
%%sql 
SELECT order_id, customer_id, pizza_id, extras
FROM customer_orders
WHERE extras IS NOT 'null' AND extras IS NOT NULL AND extras IS NOT ''

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,pizza_id,extras
5,104,1,1
7,105,2,1
9,103,1,"1, 5"
10,104,1,"1, 4"


In [35]:
# /*IN PROGRESS*/
## answer should be bacon!

## Question C3: What was the most common exclusion?

In [36]:
%%sql 
SELECT order_id, customer_id, pizza_id, exclusions
FROM customer_orders
WHERE exclusions IS NOT 'null' AND exclusions IS NOT NULL AND exclusions IS NOT ''

-- In Progress

 * sqlite:///pizza_runner.db
Done.


order_id,customer_id,pizza_id,exclusions
4,103,1,4
4,103,1,4
4,103,2,4
9,103,1,4
10,104,1,"2, 6"


In [37]:
#answer should be cheese

## Question C4: Generate an order item for each record in the customers_orders table in the format of one of the following:
* Meat Lovers
* Meat Lovers - Exclude Beef
* Meat Lovers - Extra Bacon
* Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

## Question C5: Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients
For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

## Question C6: What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

# D. Pricing and Ratings

## Question D1: If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

In [38]:
%%sql
SELECT run.runner_id, SUM(CASE WHEN cust.pizza_id = 1 THEN 12 WHEN cust.pizza_id = 2 THEN 10 END) AS total_sum
FROM customer_orders as cust
LEFT JOIN runner_orders as run ON cust.order_id = run.order_id
LEFT JOIN pizza_names ON cust.pizza_id = pizza_names.pizza_id
WHERE run.cancellation IS NULL OR run.cancellation IS 'null' OR run.cancellation IS ''
GROUP BY run.runner_id

 * sqlite:///pizza_runner.db
Done.


runner_id,total_sum
1,70
2,56
3,12


## Question D2:  What if there was an additional $1 charge for any pizza extras?

In [39]:
%%sql
SELECT run.runner_id,
    SUM(CASE WHEN cust.pizza_id = 1 THEN 12 WHEN cust.pizza_id = 2 THEN 10 END + 
        CASE WHEN cust.extras IS NOT 'null' AND cust.extras IS NOT NULL AND cust.extras IS NOT '' THEN 1 ELSE 0 END 
       ) AS total_sum
FROM customer_orders as cust
LEFT JOIN runner_orders as run ON cust.order_id = run.order_id
LEFT JOIN pizza_names ON cust.pizza_id = pizza_names.pizza_id
WHERE run.cancellation IS NULL OR run.cancellation IS 'null' OR run.cancellation IS ''
GROUP BY run.runner_id

 * sqlite:///pizza_runner.db
Done.


runner_id,total_sum
1,71
2,57
3,13


* Add cheese is $1 extra

In [40]:
%%sql
SELECT run.runner_id,
    SUM(CASE WHEN cust.pizza_id = 1 THEN 12 WHEN cust.pizza_id = 2 THEN 10 END + 
        CASE WHEN cust.extras IS NOT 'null' AND cust.extras IS NOT NULL AND cust.extras IS NOT '' THEN 1 ELSE 0 END +
        CASE WHEN cust.extras LIKE '%4%' THEN 1 ELSE 0 END
       ) AS total_sum
FROM customer_orders as cust
LEFT JOIN runner_orders as run ON cust.order_id = run.order_id
LEFT JOIN pizza_names ON cust.pizza_id = pizza_names.pizza_id
WHERE run.cancellation IS NULL OR run.cancellation IS 'null' OR run.cancellation IS ''
GROUP BY run.runner_id

 * sqlite:///pizza_runner.db
Done.


runner_id,total_sum
1,72
2,57
3,13


## Question D3: The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

## Question D4: Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?
- customer_id
- order_id
- runner_id
- rating
- order_time
- pickup_time
- Time between order and pickup
- Delivery duration
- Average speed
- Total number of pizzas

## Question D5: If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

In [77]:
%%sql
WITH cost_pizzas AS (
    SELECT run.runner_id, SUM(CASE WHEN cust.pizza_id = 1 THEN 12 WHEN cust.pizza_id = 2 THEN 10 END) AS total_sum
    FROM customer_orders as cust
    LEFT JOIN runner_orders as run ON cust.order_id = run.order_id
    LEFT JOIN pizza_names ON cust.pizza_id = pizza_names.pizza_id
    WHERE run.cancellation IS NULL OR run.cancellation IS 'null' OR run.cancellation IS ''
    GROUP BY run.runner_id
),
travel_fee AS (
    SELECT *, run.runner_id, SUM(TRIM(REPLACE(distance, 'km', '')))*0.30 as travel_profit
    FROM runner_orders AS run 
    WHERE run.cancellation IS NULL OR run.cancellation IS 'null' OR run.cancellation IS ''
    GROUP BY run.runner_id
)
SELECT travel_fee.runner_id, ROUND(SUM(travel_fee.travel_profit + cost_pizzas.total_sum), 2) as total
FROM travel_fee
LEFT JOIN cost_pizzas ON travel_fee.runner_id = cost_pizzas.runner_id
GROUP BY travel_fee.runner_id

 * sqlite:///pizza_runner.db
Done.


runner_id,total
1,89.02
2,77.54
3,15.0


In [78]:
# not sure if I'm interpreting the question correctly (something about money left over)

# E. Bonus Questions
## Question E1: If Danny wants to expand his range of pizzas - how would this impact the existing data design? Write an INSERT statement to demonstrate what would happen if a new Supreme pizza with all the toppings was added to the Pizza Runner menu?